In [1]:
# Стандартні бібліотеки
import os
from typing import Any

# Зовнішні бібліотеки
import keras
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from IPython.display import display
from keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from keras.layers import Add, Conv2D
from keras.models import load_model
from keras.preprocessing.image import array_to_img, img_to_array
from keras.utils import image_dataset_from_directory
from mpl_toolkits.axes_grid1.inset_locator import mark_inset, zoomed_inset_axes

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
print(f'TensorFlow version: {tf.__version__}')

TensorFlow version: 2.15.0


In [4]:
print(f"Доступні GPU: {tf.config.list_physical_devices('GPU')}")

Доступні GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
channels = 1
cropped_width = 498
cropped_height = 300
upscale_factor = 3

input_width = cropped_width // upscale_factor
input_height = cropped_height  // upscale_factor

TEST_FOLDER_PATH  = 'D:\\KPI\\Bachelor_thesis\\code\\data\\images\\test'
TRAIN_FOLDER_PATH = 'D:\\KPI\\Bachelor_thesis\\code\\data\\images\\train'
VAL_FOLDER_PATH   = 'D:\\KPI\\Bachelor_thesis\\code\\data\\images\\val'

RAW_TEST_FOLDER_PATH  = 'D:\\KPI\\Bachelor_thesis\\code\\data\\raw_images\\test'
RAW_TRAIN_FOLDER_PATH = 'D:\\KPI\\Bachelor_thesis\\code\\data\\raw_images\\train'
RAW_VAL_FOLDER_PATH   = 'D:\\KPI\\Bachelor_thesis\\code\\data\\raw_images\\val'

TEST_FOLDER_PATH  = '/content/drive/MyDrive/Bachelor_thesis/code/data/images/test'
TRAIN_FOLDER_PATH = '/content/drive/MyDrive/Bachelor_thesis/code/data/images/new_train'
VAL_FOLDER_PATH   = '/content/drive/MyDrive/Bachelor_thesis/code/data/images/val'

TRAIN_FOLDER_PATH = '/content/drive/MyDrive/Bachelor_thesis/content/drive/MyDrive/Bachelor_thesis/code/data/images/train_2'
VAL_FOLDER_PATH   = '/content/drive/MyDrive/Bachelor_thesis/content/drive/MyDrive/Bachelor_thesis/code/data/images/val_2'

In [6]:
# !unzip -n -d /content/drive/MyDrive/Bachelor_thesis /content/drive/MyDrive/Bachelor_thesis/val.zip

In [7]:
# print(f'Тестовий набір даних:')
# test_set = image_dataset_from_directory(TEST_FOLDER_PATH, image_size=(cropped_width, cropped_height),
#                                         batch_size=32, label_mode=None)
print(f'\nНавчальний набір даних:')
train_set = image_dataset_from_directory(TRAIN_FOLDER_PATH, image_size=(cropped_width, cropped_height),
                                         batch_size=32, label_mode=None)
print(f'\nВалідаційний набір даних:')
val_set = image_dataset_from_directory(VAL_FOLDER_PATH, image_size=(cropped_width, cropped_height),
                                       batch_size=32, label_mode=None)


Навчальний набір даних:
Found 4250 files belonging to 1 classes.

Валідаційний набір даних:
Found 750 files belonging to 1 classes.


In [8]:
def scaling(input_image):
    input_image = input_image / 255.0
    return input_image

In [9]:
# test_set = test_set.map(scaling)
train_set = train_set.map(scaling)
val_set = val_set.map(scaling)

In [10]:
# counter = 0
# for batch in train_set.take(1):
#     for img in batch:
#         if counter < 3:
#             display(array_to_img(img))
#             counter += 1
#         else:
#             break

In [11]:
def process_features(input, new_width, new_height):
    input = tf.image.rgb_to_yuv(input)
    last_axis = len(input.shape) -1
    y, u, v = tf.split(input, 3, axis=last_axis)
    return tf.image.resize(y, [new_width, new_height], method="area")


def process_target(input):
    input = tf.image.rgb_to_yuv(input)
    last_axis = len(input.shape) -1
    y, u, v = tf.split(input, 3, axis=last_axis)
    return y

In [12]:
# test_set = test_set.map(lambda x: (process_features(x, input_width, input_height),process_target(x)))
train_set = train_set.map(lambda x: (process_features(x, input_width, input_height),process_target(x)))
val_set = val_set.map(lambda x: (process_features(x, input_width, input_height),process_target(x)))

In [13]:
# counter = 0
# for batch in train_set.take(1):
#     for img_lr, img_hr in zip(batch[0], batch[1]):
#         if counter < 3:
#             _fig, ax = plt.subplots(1, 2)
#             ax[0].imshow(array_to_img(img_lr), cmap='gray')
#             ax[0].set_title('Low resolution')
#             ax[1].imshow(array_to_img(img_hr), cmap='gray')
#             ax[1].set_title('High resolution')
#             plt.show()
#             counter += 1
#         else:
#             break


In [ ]:
def rdb_block(inputs: tf.Tensor, numLayers: int) -> tf.Tensor:
    """
    Функція створює блок RDB (Residual Dense Block) для обробки вхідних даних за допомогою кількох шарів Conv2D.

    Параметри:
    inputs (tf.Tensor): Вхідний тензор з даними.
    numLayers (int): Кількість шарів Conv2D для процесу екстракції ознак RDB.

    Повертає:
    tf.Tensor: Вихідний тензор після обробки RDB блоком.
    """
    
    channels = inputs.get_shape()[-1]  #  Отримуємо кількість каналів у наших даних, що дорівнює 1
    storedOutputs = [inputs]

    #  "numLayers" представляє кількість шарів Conv2D, які використовуються для процесу екстракції ознак RDB
    for _ in range(numLayers):
        localConcat = tf.concat(storedOutputs, axis=-1)
        out = Conv2D(filters=channels, kernel_size=3, padding="same", activation="relu")(localConcat)
        storedOutputs.append(out)  #  Виходи кожного шару Conv2D додаються

    finalConcat = tf.concat(storedOutputs, axis=-1)
    #  Цей шар Conv2D називається шаром «точкової» згортки. Він в основному готує дані 
    #  для додавання до початкового входу і виходу з блоку RDB для входу в наступний шар в CNN
    finalOut = Conv2D(filters=channels, kernel_size=1, padding="same", activation="relu")(finalConcat)
    finalOut = Add()([finalOut, inputs])
    return finalOut


In [14]:
def rdb_block(inputs, numLayers):

    channels = inputs.get_shape()[-1]      # Get the amount of channels in our data, which is 1.

    storedOutputs = [inputs]

    for _ in range(numLayers):             # Here, "numLayers" represents the number of Conv2D layers
                                           # that are used for the RDB feature extraction process.
        localConcat = tf.concat(storedOutputs, axis=-1)

        out = Conv2D(filters=channels, kernel_size=3, padding="same",
            activation="tanh")(localConcat)

        storedOutputs.append(out)               # The outputs of each Conv2D layer are appended.

    finalConcat = tf.concat(storedOutputs, axis=-1)
    finalOut = Conv2D(filters=channels, kernel_size=1,    # This Conv2D layer is called "pointwise"
        padding="same", activation="tanh")(finalConcat)   # convolution layer. It basically prepares
                                                          # the data to be added to the original input
    finalOut = Add()([finalOut, inputs])                  # and exit the RDB block to enter the next
                                                          # layer in the CNN.
    return finalOut

In [15]:
def Model(channels, upscale_factor):

    inputs = keras.Input(shape=(None, None, channels))
    X = Conv2D(64, 5, padding='same', activation='tanh', kernel_initializer='Orthogonal')(inputs)
    X = Conv2D(64, 3, padding='same', activation='tanh', kernel_initializer='Orthogonal')(X)
    X = rdb_block(X, numLayers=3)
    X = Conv2D(32, 3, padding='same', activation='tanh', kernel_initializer='Orthogonal')(X)
    X = rdb_block(X, numLayers=3)
    X = Conv2D(channels * (upscale_factor**2), 3, padding='same', activation='tanh', kernel_initializer='Orthogonal')(X)

    outputs = tf.nn.depth_to_space(X, upscale_factor)
    return keras.Model(inputs, outputs)

In [16]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, min_delta=0.0001)
model = Model(channels, upscale_factor)
                                               # Adam optimizer due to its efficiency, MSE
model.compile(optimizer='adam', loss='MSE')    # loss function because it's a regression model.

model.summary()                                # Show a summary of the layers in our model.

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, None, 1)]      0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, None, None, 64)       1664      ['input_1[0][0]']             
                                                                                                  
 conv2d_1 (Conv2D)           (None, None, None, 64)       36928     ['conv2d[0][0]']              
                                                                                                  
 tf.identity (TFOpLambda)    (None, None, None, 64)       0         ['conv2d_1[0][0]']            
                                                                                              

In [17]:
checkpoint_callback = ModelCheckpoint('RDB_Tanh_best.h5', monitor='val_loss', save_best_only=True, mode='min')
csv_logger = CSVLogger('training_log.csv', append=True)

In [18]:
model.fit(train_set, epochs=100, callbacks= [early_stopping], validation_data = val_set)

Epoch 1/100
133/133 [==============================] - 341s 2s/step - loss: 0.0329 - val_loss: 0.0058
Epoch 2/100
133/133 [==============================] - 58s 420ms/step - loss: 0.0047 - val_loss: 0.0043
Epoch 3/100
133/133 [==============================] - 58s 421ms/step - loss: 0.0042 - val_loss: 0.0036
Epoch 4/100
133/133 [==============================] - 58s 428ms/step - loss: 0.0033 - val_loss: 0.0033
Epoch 5/100
133/133 [==============================] - 59s 434ms/step - loss: 0.0033 - val_loss: 0.0031
Epoch 6/100
133/133 [==============================] - 61s 446ms/step - loss: 0.0031 - val_loss: 0.0030
Epoch 7/100
133/133 [==============================] - 59s 433ms/step - loss: 0.0031 - val_loss: 0.0031
Epoch 8/100
133/133 [==============================] - 58s 422ms/step - loss: 0.0031 - val_loss: 0.0029
Epoch 9/100
133/133 [==============================] - 58s 417ms/step - loss: 0.0031 - val_loss: 0.0030
Epoch 10/100
133/133 [==============================] - 60s 435ms/

In [19]:
model.save('RDB_Tanh.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
model.save('RDB_Tanh.keras')

In [21]:
model.save_weights('RDB_Tanh_weights.h5')